In [1]:
#### 패키지 임포트
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import os
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
pd.set_option('display.max_columns', None)

In [2]:
#### 본문to형태소(morphs) 파일
df = pd.read_csv("../dataset/2022/본문_형태소분석/나무문화_본문_8월_형태소분석.txt", encoding='UTF-8')
df['morphs']=df['morphs'].apply(lambda x : x.strip("'][").split("', '"))
print(df.shape, df.columns)
docs = df['morphs'].to_list()
print(len(docs))

(1021057, 4) Index(['split_str', 'org_idx', 'morphs', 'pos_tag'], dtype='object')
1021057


In [3]:
df

,split_str,org_idx,morphs,pos_tag
0,50m NAVER Corp 송천도서관 전라북도 전주시 덕진구 솔내로 212 송천도서...,0,"[50, m, NAVER, Corp, 송천, 도서관, 전라북도, 전주시, 덕진구, ...","[('50', 'Number'), ('m', 'Alpha'), ('NAVER', '..."
1,강사는 불편한편의점 의 김호연 작가이다 불편한 편의점 2 저자 김호연 출판 나무옆의...,0,"[강사, 는, 불편한, 편의점, 의, 김호연, 작가, 이다, 불편한, 편의점, 2,...","[('강사', 'Noun'), ('는', 'Josa'), ('불편하다', 'Adje..."
2,주제 소설가의 삶과 문화콘텐츠 스토리텔링 강의 시작전 22년간 영화 만화 출판 관련...,0,"[주제, 소설가, 의, 삶, 과, 문화, 콘텐츠, 스토리텔링, 강의, 시, 작전, ...","[('주제', 'Noun'), ('소설가', 'Noun'), ('의', 'Josa'..."
3,문화콘텐츠 문화콘텐츠란 인간의 감성 창의성 상상력을 원천으로 한 문화상품 문화콘텐츠...,0,"[문화, 콘텐츠, 문화, 콘텐츠, 란, 인간, 의, 감성, 창의성, 상상력, 을, ...","[('문화', 'Noun'), ('콘텐츠', 'Noun'), ('문화', 'Noun..."
4,영화 드라마 만화 대중음악 뮤지컬 모바일게임 인터넷 모바일 테마파크 전시 축제 공간...,0,"[영화, 드라마, 만화, 대중음악, 뮤지컬, 모바일, 게임, 인터넷, 모바일, 테마...","[('영화', 'Noun'), ('드라마', 'Noun'), ('만화', 'Noun..."
...,...,...,...,...
1021052,라벨에 BIO라는 단어가 지워져 있습니다,20813,"[라벨, 에, BIO, 라는, 단어, 가, 지워져, 있습니다]","[('라벨', 'Noun'), ('에', 'Josa'), ('BIO', 'Alpha..."
1021053,원재료의 일부로 들어간 유기농 설탕과 시나몬이 유럽에서는 유기농 인증서가 있음에도 ...,20813,"[원, 재료, 의, 일부, 로, 들어간, 유기농, 설탕, 과, 시나몬, 이, 유럽,...","[('원', 'Modifier'), ('재료', 'Noun'), ('의', 'Jos..."
1021054,아버지를 아버지라 부르지 못하는 홍길동처럼 유기농을 유기농이라 부르지 못하는 홍길동...,20813,"[아버지, 를, 아버지, 라, 부르지, 못, 하는, 홍길동, 처럼, 유기농, 을, ...","[('아버지', 'Noun'), ('를', 'Josa'), ('아버지', 'Noun..."
1021055,무더운 여름 건강 조심하십시요,20813,"[무더운, 여름, 건강, 조심하십시요]","[('무덥다', 'Adjective'), ('여름', 'Noun'), ('건강', ..."


In [4]:
#### 각각의 본문이 str타입('나는 어제 밥을 먹었다')일 경우 list(['나', '는', '어제', '밥', '을' '먹다', '이다'])로 바꿔줌
docs2 = docs.copy()
# for i in tqdm(range(len(docs))):
#     docs2[i] = docs2[i].split()

In [5]:
#### 명사 빈도 상위 50개 단어 파일
topwords_df = pd.read_csv("../dataset/2022/본문_형태소분석/8월/나무문화_8월_형태소분석(명사).csv", encoding='UTF-8')
print(topwords_df.shape, topwords_df.columns)
topwords50 = topwords_df['명사'].to_list()[:50]
print(len(topwords50))
print(topwords50)

(163845, 2) Index(['명사', '명사빈도'], dtype='object')
50
['것', '수', '그', '이', '곳', '사람', '때', '등', '우리', '말', '생각', '나', '더', '내', '문화', '시간', '의', '나무', '길', '위', '사진', '안', '중', '때문', '전', '제', '여행', '하나', '일', '집', '마을', '공간', '아이', '날', '저', '위해', '카페', '개', '작품', '모습', '마음', '시작', '다른', '층', '속', '앞', '또', '대한', '거', '책']


In [6]:
#### 중복 단어 제거하여 리스트 생성
unique_lines = [list(set(line)) for line in tqdm(docs2)]

100%|██████████| 1021057/1021057 [00:06<00:00, 159949.87it/s]


In [7]:
#### 중복 제거 잘 됐나 개수로 확인
print(len(docs[0]))
print(len(unique_lines[0]))

35
31


In [8]:
#### 전체 단어쌍 빈도 dict 형식으로 생성
freq_count = {}   #동시출현 빈도가 저장될 dict
for words in tqdm(unique_lines):
    # 전체 단어에 대한 동시 출현 빈도
    for i, a in enumerate(words):
        for b in words[i+1:]:
            if a == b: continue
            elif a>b:
                freq_count[b, a] = freq_count.get((b, a),0) + 1
            else :
                freq_count[a, b] = freq_count.get((a, b),0) + 1

100%|██████████| 1021057/1021057 [12:55<00:00, 1317.40it/s]


In [9]:
#### 딕셔너리를 데이터프레임에 담기
tt_freq_df=pd.DataFrame.from_dict(freq_count, orient='index')

In [10]:
#### 딕트to데이터프레임 저장
tt_freq_df.to_csv("../dataset/2022/단어동시출현행렬/8월/단어동시출현_딕셔너리_8월.csv", encoding='UTF-8')

In [11]:
#### dict형식을 컬럼명 지정해서 데이터프레임으로 만들기
list1 = [(tt_freq_df.index[i][0], tt_freq_df.index[i][1], tt_freq_df[0][i]) for i in tqdm(range(len(tt_freq_df)))]
freq_df2 = pd.DataFrame(list1, columns=["term1","term2","freq"])
print(freq_df2.shape)
freq_df2.head()

100%|██████████| 193375274/193375274 [23:16<00:00, 138435.51it/s]


(193375274, 3)


,term1,term2,freq
0,만남,와의,161
1,2022년,만남,34
2,만남,작가,104
3,만남,행사,36
4,만남,전라북도,3


In [12]:
#### 컬럼명 지정한 dict_to_데이터프레임 저장
freq_df2.to_csv("../dataset/2022/단어동시출현행렬/8월/단어동시출현_df_8월.csv", index=False, encoding='UTF-8')

In [13]:
#### 50위 이내에 속한 단어들로 이루어진 Dataframe을 추출
list2 =[(freq_df2.loc[i][0], freq_df2.loc[i][1], freq_df2.loc[i][2]) for i in tqdm(range(len(freq_df2))) if (freq_df2['term1'][i] in topwords50) &(freq_df2['term2'][i] in topwords50)]
freq_df3 = pd.DataFrame(list2, columns=["term1","term2","freq"])
freq_df3

100%|██████████| 193375274/193375274 [39:28<00:00, 81651.90it/s]


,term1,term2,freq
0,날,시작,816
1,날,의,8405
2,날,문화,1161
3,날,내,2079
4,시작,의,11535
...,...,...,...
1220,날,층,423
1221,때문,카페,343
1222,길,대한,430
1223,공간,대한,558


In [14]:
#### 상위50위 단어들로 이루어진 단어쌍 데이터프레임 저장
freq_df3.to_csv("../dataset/2022/단어동시출현행렬/8월/단어동시출현_df_상위50_8월.csv", index=False, encoding='UTF-8')

In [15]:
#### 공출현빈도(1-mode matrix) 만들기
word_co_matx = np.zeros((50,50))
mat_idx = topwords50 # 단어동시출현행렬이니까 인덱스와 칼럼을 동일하게 지정
mat_col = topwords50
word_co_matx = pd.DataFrame(word_co_matx, index=mat_idx, columns=mat_col)
print(word_co_matx.shape)


#### 단어쌍의 첫 번째 단어가 인덱스와 같다 and 두 번째 단어가 칼럼과 같다 → 단어쌍빈도를 해당 셀 값에 삽입
for n in tqdm(range(len(freq_df3))):
    for idx in mat_idx:
        if idx == freq_df3['term1'][n]:
            for col in mat_col:
                if col == freq_df3['term2'][n]:
                    word_co_matx.loc[idx, col] = freq_df3['freq'][n]
                    word_co_matx.loc[col, idx] = freq_df3['freq'][n]

(50, 50)


100%|██████████| 1225/1225 [00:00<00:00, 1668.51it/s]


In [16]:
#### 1-mode 매트릭스 확인하기
word_co_matx

,것,수,그,이,곳,사람,때,등,우리,말,생각,나,더,내,문화,시간,의,나무,길,위,사진,안,중,때문,전,제,여행,하나,일,집,마을,공간,아이,날,저,위해,카페,개,작품,모습,마음,시작,다른,층,속,앞,또,대한,거,책
것,0.0,30783.0,30132.0,115734.0,10572.0,17903.0,14916.0,8987.0,13249.0,15075.0,13434.0,15307.0,10603.0,12024.0,5768.0,6569.0,86466.0,5449.0,4371.0,8275.0,4114.0,8477.0,7804.0,9934.0,7146.0,5861.0,3411.0,6603.0,9703.0,4298.0,2366.0,3385.0,4005.0,4217.0,6130.0,5808.0,1960.0,3628.0,3771.0,4499.0,6175.0,4696.0,6850.0,1813.0,6671.0,4809.0,5086.0,6603.0,3742.0,4035.0
수,30783.0,0.0,17010.0,71378.0,12685.0,10892.0,8970.0,10888.0,8456.0,7826.0,7568.0,10303.0,7775.0,7568.0,6900.0,6834.0,62456.0,5526.0,4717.0,5962.0,4114.0,5702.0,5812.0,6594.0,5970.0,5074.0,3502.0,4155.0,6313.0,3295.0,2897.0,6369.0,4474.0,3128.0,3696.0,3902.0,2615.0,3472.0,3745.0,4281.0,4117.0,3045.0,4634.0,3085.0,4994.0,3683.0,3431.0,4416.0,2607.0,3190.0
그,30132.0,17010.0,0.0,64389.0,9118.0,12802.0,10611.0,6017.0,7651.0,9390.0,6825.0,9750.0,5546.0,7426.0,3656.0,3678.0,61623.0,3858.0,3333.0,5783.0,2349.0,5500.0,6903.0,5740.0,5129.0,4096.0,1785.0,4147.0,5881.0,3287.0,1758.0,2055.0,2037.0,3020.0,3137.0,3760.0,1195.0,2435.0,2856.0,3520.0,3903.0,3154.0,3918.0,1163.0,5130.0,3479.0,3621.0,4139.0,2521.0,2624.0
이,115734.0,71378.0,64389.0,0.0,48017.0,44914.0,37727.0,30572.0,26881.0,29629.0,27437.0,34964.0,23833.0,27062.0,17861.0,20490.0,216457.0,18147.0,15482.0,19345.0,15393.0,22605.0,21163.0,20232.0,21032.0,17557.0,10157.0,15042.0,19708.0,13795.0,10899.0,13834.0,12534.0,15457.0,16028.0,13342.0,8974.0,12321.0,11486.0,14475.0,14532.0,13265.0,14686.0,8585.0,15083.0,13450.0,12748.0,12720.0,12891.0,10998.0
곳,10572.0,12685.0,9118.0,48017.0,0.0,5228.0,4790.0,4386.0,3387.0,2717.0,3175.0,4338.0,3293.0,3746.0,3809.0,3569.0,27935.0,3161.0,4141.0,2588.0,3404.0,3308.0,3925.0,2150.0,3241.0,3062.0,4250.0,2246.0,2233.0,2293.0,3616.0,3036.0,3199.0,2050.0,2862.0,1841.0,2864.0,1899.0,913.0,2250.0,2010.0,1730.0,2740.0,1634.0,1944.0,2212.0,1950.0,932.0,1411.0,955.0
사람,17903.0,10892.0,12802.0,44914.0,5228.0,0.0,5974.0,3063.0,4765.0,6020.0,5330.0,6868.0,3779.0,4716.0,2416.0,2558.0,31266.0,2349.0,2330.0,2915.0,2039.0,3720.0,3211.0,3427.0,2751.0,2050.0,1311.0,2339.0,4053.0,2153.0,1699.0,1465.0,1125.0,1974.0,2346.0,2574.0,1017.0,1345.0,1122.0,2030.0,2795.0,1781.0,3823.0,840.0,2612.0,2217.0,2225.0,1945.0,2001.0,1691.0
때,14916.0,8970.0,10611.0,37727.0,4790.0,5974.0,0.0,3193.0,4302.0,4522.0,4168.0,6063.0,3425.0,4733.0,1753.0,2624.0,26355.0,2123.0,1859.0,2531.0,2033.0,3522.0,2845.0,2672.0,2922.0,2715.0,1773.0,2187.0,3482.0,2292.0,1062.0,988.0,1583.0,2155.0,2445.0,1918.0,849.0,1423.0,924.0,1761.0,2173.0,2068.0,2095.0,815.0,1937.0,1880.0,2051.0,1496.0,1909.0,1428.0
등,8987.0,10888.0,6017.0,30572.0,4386.0,3063.0,3193.0,0.0,1976.0,2512.0,1558.0,4258.0,1787.0,2742.0,4518.0,1936.0,30922.0,2738.0,1609.0,3258.0,1370.0,1757.0,3213.0,1701.0,3230.0,3139.0,1140.0,1364.0,2394.0,1424.0,1645.0,2357.0,1210.0,883.0,1091.0,2037.0,1020.0,2545.0,1914.0,1511.0,907.0,1486.0,1530.0,1282.0,1908.0,1302.0,1109.0,2210.0,546.0,1319.0
우리,13249.0,8456.0,7651.0,26881.0,3387.0,4765.0,4302.0,1976.0,0.0,3768.0,3653.0,4025.0,2991.0,3061.0,2478.0,2418.0,24053.0,1667.0,1748.0,2203.0,1614.0,2807.0,2071.0,2227.0,2012.0,1481.0,1219.0,1815.0,2678.0,2385.0,965.0,1001.0,1737.0,1666.0,1561.0,1855.0,823.0,1074.0,875.0,1341.0,2033.0,1387.0,1746.0,710.0,2389.0,1682.0,1607.0,1657.0,1569.0,1283.0
말,15075.0,7826.0,9390.0,29629.0,2717.0,6020.0,4522.0,2512.0,3768.0,0.0,3267.0,5002.0,2362.0,4099.0,1433.0,1359.0,24200.0,1441.0,1198.0,2183.0,907.0,2746.0,2246.0,2138.0,2114.0,1943.0,664.0,1790.0,2798.0,1296.0,629.0,620.0,997.0,1290.0,1599.0,1479.0,419.0,1119.0,689.0,1139.0,1887.0,1223.0,1750.0,365.0,1893.0,1423.0,1512.0,1520.0,1424.0,1181.0


In [17]:
#### 공출현빈도 데이터프레임 저장하기~~
#### ※ 인덱스가 저장돼야 하므로 index 파라미터 지정하지 말 것.
word_co_matx.to_csv("../dataset/2022/단어동시출현행렬/8월/공출현빈도_df_8월.csv", encoding='UTF-8')